In [83]:
import ipywidgets as widgets
from ipywidgets import interactive_output, interact
from menu import init_widgets, style, setup_menu, select_table
from dashboard import (
    Seaborn_Barplot,
    Seaborn_Boxplot,
    Seaborn_Scatter,
    Boxplot_ggplot,
    Histogram_ggplot,
)
from IPython.display import display, clear_output
import pandas as pd
import numpy as np
from itables import init_notebook_mode, show

In [84]:
%%capture

%load_ext sql

%sql duckdb:///../data/database/car_data.duckdb

%config SqlMagic.displaycon = False

In [85]:
years = %sql select DISTINCT(model_year) from all_vehicles
years = tuple([model_year[0] for model_year in years])

In [86]:
makes = %sql select DISTINCT(make) from all_vehicles
makes = [m[0] for m in makes]

In [87]:
classes = %sql select DISTINCT(vehicle_class) from all_vehicles
classes = [c[0] for c in classes]

In [88]:
co2 = %sql select DISTINCT(co2_rating) from all_vehicles where co2_rating is not null
co2 = [c[0] for c in co2]
# convert to int
co2 = [int(c) for c in co2]

In [89]:
vehicle_type = %sql select DISTINCT(vehicle_type) from all_vehicles
vehicle_type = [v[0] for v in vehicle_type]

## Interactive Table
> Multiple values for Vehcile Class can be selected with shift and/or ctrl (or command) pressed and mouse clicks or arrow keys.

In [90]:
def select_table_electric(vehicle_type, year, vehicle_class, make, co2):
    """
    Select table based on vehicle type
    
    Parameters
    ----------
    vehicle_type : str
        Vehicle type (fuel-only, hybrid, or electric)
    year : int
        Model year
    vehicle_class : list
        Vehicle class (compact, midsize, etc.)
    make : str
        Car manufacturer
    co2 : int
        CO2 rating (higher rating suggests lower CO2 emissions)
    """
    print("Performaing query")
    query = select_table(vehicle_type, year, vehicle_class, make, co2)

    # Use JupySQL magic to execute the query
    result = %sql {{query}}

    # Convert the result to a Pandas DataFrame
    df = result.DataFrame()

    clear_output(wait=True)

    show(df, classes="display nowrap compact")

(
    widget_vehicle_type,
    widget_year,
    widget_make,
    widget_vehicle_class,
    widget_co2,
) = init_widgets(years, makes, classes, vehicle_type, style)

tab = setup_menu(
    widget_vehicle_type, widget_year, widget_vehicle_class, widget_make, widget_co2
)

output = interactive_output(
    select_table_electric,
    {
        "vehicle_type": widget_vehicle_type,
        "year": widget_year,
        "vehicle_class": widget_vehicle_class,
        "make": widget_make,
        "co2": widget_co2,
    },
)

display(tab, output)

Output()

## Plots

This dashboard contains 5 plots, created using the `ggplot` API, `seaborn`, and `ipywidgets`:

1. Bar Plot of Car Manufacturing Trends
2. Boxplot of Fuel Consumption and GHG emissions for All Vehicle Types
3. Scatter Plot of Electric Vehicle Ranges and Charging Time by Car Size and Model Year
4. Histogram of GHG emissions by Vehicle and Fuel Type
5. GHG emissions of Hybrid and Fuel-Only US Car Brands by Transmission Type

## Bar plot of car manufacturing trends

In [91]:
%%capture

%%sql --save q_1_hybrid_electric --no-execute
SELECT DISTINCT model_year, vehicle_type, COUNT(id) AS num_vehicles
FROM all_vehicles
WHERE vehicle_type = 'hybrid' OR vehicle_type = 'electric'
GROUP BY model_year, vehicle_type
ORDER BY num_vehicles DESC;

In [92]:
%%capture

%%sql --save q_1_fuel --no-execute
SELECT DISTINCT model_year, vehicle_type, COUNT(id) AS num_vehicles
FROM fuel
GROUP BY model_year, vehicle_type
ORDER BY model_year;

In [93]:
%%capture

hybrid_electric_count = %sql SELECT * FROM q_1_hybrid_electric
hybrid_electric_count = pd.DataFrame(hybrid_electric_count).sort_values(by=["model_year"])

In [94]:
%%capture

fuel_count = %sql SELECT * FROM q_1_fuel
fuel_count = pd.DataFrame(fuel_count).sort_values(by=["model_year"])

In [95]:
%%capture

barplot = Seaborn_Barplot(fuel_count, hybrid_electric_count)
interact(barplot.draw_bar_year_count, data=barplot.radio_button);

## Boxplot of consumption and CO2 emissions for all vehicle types

In [96]:
%%capture

%%sql --save boxplot_fuel_consum --no-execute
SELECT consumption_city_l_100_km::FLOAT as consumption_city_l_100_km, consumption_hwy_l_100_km::FLOAT as consumption_hwy_l_100_km, consumption_comb_l_100_km::FLOAT as consumption_comb_l_100_km, co2_emissions_g_km::FLOAT as co2_emissions_g_km
FROM all_vehicles

In [97]:
boxplot = Boxplot_ggplot()
interact(boxplot.fuel_co2_boxplot, columns=boxplot.selection_button);

interactive(children=(SelectMultiple(description='Column(s):', index=(1,), options=('consumption_city_l_100_km…

##  Scatter plot of electric vehicle ranges and charging time by car size and model year

In [98]:
%%capture

%%sql --save electric_range_charge --no-execute
SELECT range_km, recharge_time_h, vehicle_class, model_year
FROM electric

In [99]:
def clean_electric_columns(electric_range):
    electric_range["model_year"] = electric_range["model_year"].astype(int)

    electric_range["vehicle_size"] = np.where(electric_range["vehicle_class"].isin(
        ["subcompact", "compact", "mid-size", "full-size", "two-seater"]
    ),
    "Sedan or smaller",
    "SUV or larger")

    electric_range["model_year_grouped"] = np.where(electric_range["model_year"] <= 2021, "2012-2021", "2022-2023")

    return electric_range


In [100]:
%%capture

electric_range = %sql SELECT * FROM electric_range_charge

electric_range = pd.DataFrame(electric_range)

electric_range = clean_electric_columns(electric_range)

In [101]:
scatter = Seaborn_Scatter(electric_range)
interact(scatter.draw_scatter_electric_range, hue=scatter.dropdown);

interactive(children=(Dropdown(description='(Un)select hue:', options=('vehicle_size', 'model_year_grouped', N…

## Histogram of CO2 emissions by vehcile and fuel type

In [102]:
%%capture

%%sql --save hist_co2 --no-execute
SELECT vehicle_type, fuel_type, co2_emissions_g_km::INTEGER as co2_emissions_g_km
FROM all_vehicles
WHERE co2_emissions_g_km is not null

In [103]:
histogram = Histogram_ggplot()
interact(
    histogram.co2_histogram,
    b=histogram.intslider,
    cmap=histogram.dropdown,
    fill=histogram.radio_button,
);

interactive(children=(IntSlider(value=10, description='Bins:', max=20, min=1), Dropdown(description='Colormap:…

## CO2 emissions of hybrid and fuel-only US car brands by transmission type

In [104]:
%%capture

%%sql --save co2_usa --no_execute
SELECT vehicle_type, make, co2_emissions_g_km, transmission_type
FROM all_vehicles
WHERE co2_emissions_g_km IS NOT null AND
vehicle_type IN ('fuel-only', 'hybrid') AND
make IN ('cadillac', 'chevrolet', 'chrysler', 'ford', 'jeep', 'lincoln')

In [105]:
%%capture

co2_usa = %sql SELECT * FROM co2_usa
co2_usa = pd.DataFrame(co2_usa)

In [106]:
boxplot = Seaborn_Boxplot(co2_usa)
interact(boxplot.draw_boxplot_usa, hue=boxplot.dropdown);

interactive(children=(Dropdown(description='(Un)select hue:', options=('vehicle_type', 'transmission_type', No…